In [1]:
import json
import numpy as np

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline

from keras.optimizers import SGD, RMSprop
from keras.applications.vgg16 import VGG16

import balloonml
import cv2

Using TensorFlow backend.


In [2]:
config ={}

config['output']='disdat-104-v1'

config['train_data_dir'] = '../clean-data'
config['validation_data_dir'] = '../clean-validation'

config['bottleneck_train'] = 'bottleneck_features_train.npy'
config['bottleneck_validation'] = 'bottleneck_features_validation.npy'

# True if bottleneck features are not available yet, 
# False if they are and can be lodaded
calculate_bottleneck = True 

config['epochs'] = 25
config['epochs-refine'] = 10

config['batch_size'] = 32

config['optimizer']='sgd'
config['lr']=0.002
config['decay']=1/100 * (config['lr'])/(100000/config['batch_size'])

with open(config['output']+'.config', 'w') as fp:
    json.dump(config, fp)

## Saving or loading bottleneck features

In [ ]:
if calculate_bottleneck:
    model = VGG16(include_top=False, weights='imagenet')
    train_data = balloonml.bottleneck(model,config['train_data_dir'],config['bottleneck_train'], config['batch_size'])
    validation_data = balloonml.bottleneck(model,config['validation_data_dir'],config['bottleneck_validation'], config['batch_size'])
else:
    train_data = np.load(config['bottleneck_train'])
    validation_data = np.load(config['bottleneck_validation'])

Found 115042 images belonging to 104 classes.


## Running model

In [ ]:
if config['optimizer']=='sgd':
    optimizer=SGD(lr=config['lr'], decay=config['decay'])
elif config['optimizer']=='rmsprop':
    optimizer=RMSprop(lr=config['lr'], decay=config['decay'])

history, model = balloonml.train_top(
    train_data, 
    validation_data,
    train_data_dir=config['train_data_dir'], 
    validation_data_dir=config['validation_data_dir'],
    optimizer=optimizer,
    batch_size = config['batch_size'],
    epochs=config['epochs'],
    output=config['output'])
balloonml.plotResult(history)

In [ ]:
refine_optimizer=SGD(lr=1e-4, momentum=0.9)

refined_history, refined_model = balloonml.finetune(
    train_data_dir=config['train_data_dir'],
    validation_data_dir=config['validation_data_dir'], 
    optimizer=refine_optimizer, 
    weights_top_layer=config['output']+'.h5', 
    batch_size=config['batch_size'], 
    epochs=config['epochs-refine'],
    output=config['output']+'-refined')
balloonml.plotResult(refined_history)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
img_width, img_height = 224, 224
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense
import keras.callbacks

base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))

datagen_top = ImageDataGenerator(rescale=1. / 255)
generator_training_top = datagen_top.flow_from_directory(
    config['train_data_dir'],
    target_size=(img_width, img_height),
    batch_size=config['batch_size'],
    class_mode='categorical',
    shuffle=False)

num_classes = len(generator_training_top.class_indices)


# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(num_classes, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(config['output']+'.h5')

# add the model on top of the convolutional base
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

In [ ]:
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:15]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [ ]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        config['train_data_dir'],
        target_size=(img_height, img_width),
        batch_size=config['batch_size'],
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        config['validation_data_dir'],
        target_size=(img_height, img_width),
        batch_size=config['batch_size'],
        class_mode='categorical')

nb_train_samples = len(train_generator.filenames)

nb_validation_samples = len(validation_generator.filenames)

tbCallBack = keras.callbacks.TensorBoard(log_dir='./logs/'+config['output']+'-refine', histogram_freq=0, write_graph=True, write_images=True)

# fine-tune the model
history = model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // config['batch_size'],
        epochs=config['epochs-refine'],
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // config['batch_size'],
        callbacks=[tbCallBack])

model.save_weights(config['output']+"-refined.h5")

In [ ]:
import coremltools

In [ ]:
coreml_model  = coremltools.converters.keras.convert(model,
                                                    image_input_names='image',
                                                    class_labels='labels.txt',
                                                    input_names=['image'],  
                                                    red_bias=-123.68,
                                                    green_bias=-116.78,
                                                    blue_bias=-103.94
)

In [ ]:
coreml_model.save(config['output']+"-refined.mlmodel")

## Prediction 1

In [ ]:
# load the class_indices saved in the earlier step
class_dictionary = np.load('class_indices.npy').item()

num_classes = len(class_dictionary)

# add the path to your test image below

orig = cv2.imread('test_images/keyboard.jpg')

print("[INFO] loading and preprocessing image...")
image = load_img(image_path, target_size=(224, 224))
image = img_to_array(image)

# important! otherwise the predictions will be '0'
image = image / 255

image = np.expand_dims(image, axis=0)



class_predicted = model.predict_classes(image)

probabilities = model.predict_proba(image)
print(probabilities)
inID = class_predicted[0]



## Prediction

In [ ]:
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))

datagen_top = ImageDataGenerator(rescale=1. / 255)
generator_training_top = datagen_top.flow_from_directory(
    config['train_data_dir'],
    target_size=(img_width, img_height),
    batch_size=config['batch_size'],
    class_mode='categorical',
    shuffle=False)

num_classes = len(generator_training_top.class_indices)


# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(num_classes, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
#top_model.load_weights(config['output']+'.h5')

top_model.load_weights(config['output']+'.h5')


# add the model on top of the convolutional base
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))
model.compile(loss='binary_crossentropy',
              optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [ ]:
model.load_weights(config['output']+'-refined.h5')

In [ ]:
image_path = 'test_images/keyboard.jpg'
img=mpimg.imread(image_path)
plt.imshow(img)
print("I think this is a "+balloonml.predict(image_path, config['output']+"-refined.h5")+".")

In [ ]:
orig = cv2.imread('test_images/keyboard.jpg')

print("[INFO] loading and preprocessing image...")
image = load_img('test_images/keyboard.jpg', target_size=(224, 224))
image = img_to_array(image)

# important! otherwise the predictions will be '0'
image = image / 255

image = np.expand_dims(image, axis=0)

In [ ]:
probabilities = model.predict(image)
print(classes)
print(probabilities)